In [1]:
!cp '/content/drive/MyDrive/HACKER EARTH/7342422066d511eb.zip'  .

In [2]:
!unzip -q 7342422066d511eb.zip

In [3]:
import pandas as pd
import numpy as np
import datetime as dt

In [4]:
train=pd.read_csv('/content/dataset/train.csv')
train.head()

,Customer Id,Artist Name,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location,Scheduled Date,Delivery Date,Customer Location,Cost
0,fffe3900350033003300,Billy Jenkins,0.26,17.0,6.0,4128.0,Brass,13.91,16.27,Yes,Yes,No,Airways,No,Working Class,No,06/07/15,06/03/15,"New Michelle, OH 50777",-283.29
1,fffe3800330031003900,Jean Bryant,0.28,3.0,3.0,61.0,Brass,6.83,15.00,No,No,No,Roadways,No,Working Class,No,03/06/17,03/05/17,"New Michaelport, WY 12072",-159.96
2,fffe3600370035003100,Laura Miller,0.07,8.0,5.0,237.0,Clay,4.96,21.18,No,No,No,Roadways,Yes,Working Class,Yes,03/09/15,03/08/15,"Bowmanshire, WA 19241",-154.29
3,fffe350031003300,Robert Chaires,0.12,9.0,NaN,NaN,Aluminium,5.81,16.31,No,No,No,NaN,No,Wealthy,Yes,05/24/15,05/20/15,"East Robyn, KY 86375",-161.16
4,fffe3900320038003400,Rosalyn Krol,0.15,17.0,6.0,324.0,Aluminium,3.18,11.94,Yes,Yes,Yes,Airways,No,Working Class,No,12/18/16,12/14/16,"Aprilside, PA 52793",-159.23


In [89]:
test=pd.read_csv('/content/dataset/test.csv')
test.head()

,Customer Id,Artist Name,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location,Scheduled Date,Delivery Date,Customer Location
0,fffe3400310033003300,James Miller,0.35,53.0,18.0,871.0,Wood,5.98,19.11,Yes,Yes,No,Airways,No,Working Class,No,07/03/17,07/06/17,"Santoshaven, IA 63481"
1,fffe3600350035003400,Karen Vetrano,0.67,7.0,4.0,108.0,Clay,6.92,13.96,No,No,No,Roadways,Yes,Working Class,No,05/02/16,05/02/16,"Ericksonton, OH 98253"
2,fffe3700360030003500,Roseanne Gaona,0.61,6.0,5.0,97.0,Aluminium,4.23,13.62,Yes,No,No,Airways,No,Working Class,No,01/04/18,01/06/18,APO AP 83453
3,fffe350038003600,Todd Almanza,0.14,15.0,8.0,757.0,Clay,6.28,23.79,No,Yes,No,Roadways,Yes,Wealthy,No,09/14/17,09/17/17,"Antonioborough, AL 54778"
4,fffe3500390032003500,Francis Rivero,0.63,10.0,4.0,1673.0,Marble,4.39,17.83,No,Yes,Yes,Roadways,No,Working Class,Yes,12/03/17,12/02/17,"Lake Frances, LA 03040"


In [6]:
train['Cost']=train['Cost'].abs()

In [7]:
train['Cost'].unique()

array([ 283.29,  159.96,  154.29, ...,  354.55, 5037.5 ,  722.47])

In [8]:
train['Cost']=np.log(train['Cost'])

In [9]:
train['Cost'].unique()

array([5.64647111, 5.07492378, 5.03883395, ..., 5.87084938, 8.52466521,
       6.5826759 ])

In [10]:
train.isnull().sum()

Customer Id                 0
Artist Name                 0
Artist Reputation         750
Height                    375
Width                     584
Weight                    587
Material                  764
Price Of Sculpture          0
Base Shipping Price         0
International               0
Express Shipment            0
Installation Included       0
Transport                1392
Fragile                     0
Customer Information        0
Remote Location           771
Scheduled Date              0
Delivery Date               0
Customer Location           0
Cost                        0
dtype: int64

**Impting Missing values**

In [11]:
def fillNanValues(df, col, value):
    df[col].fillna(value, inplace=True)

In [13]:
fillNanValues(train, 'Artist Reputation', train['Artist Reputation'].mean())
fillNanValues(test, 'Artist Reputation', test['Artist Reputation'].mean())
print(train['Artist Reputation'].isna().any())#any returns boolean values
print(test['Artist Reputation'].isna().any())

False
False


In [14]:
fillNanValues(train, 'Height', train['Height'].mean())
print(train['Height'].isna().sum())
fillNanValues(test, 'Height', test['Height'].mean())
print(test['Height'].isna().sum()) # sum gives numerival assurance

0
0


In [15]:
fillNanValues(train, 'Width', train['Width'].mean())
print(train['Width'].isna().sum())
fillNanValues(test, 'Width', test['Width'].mean())
print(test['Width'].isna().sum())

0
0


In [16]:
fillNanValues(train, 'Weight', train['Weight'].mean())
print(train['Weight'].isna().sum())
fillNanValues(test, 'Weight', test['Weight'].mean())
print(test['Weight'].isna().sum())

0
0


In [17]:
train['Transport'].fillna("missingTransport",inplace=True)
train['Material'].fillna("missingMaterial",inplace=True)
train['Remote Location'].fillna("missingRemoteLocation",inplace=True)

In [18]:
test['Transport'].fillna("missingTransport",inplace=True)
test['Material'].fillna("missingMaterial",inplace=True)
test['Remote Location'].fillna("missingRemoteLocation",inplace=True)

In [19]:
train.isnull().sum()

Customer Id              0
Artist Name              0
Artist Reputation        0
Height                   0
Width                    0
Weight                   0
Material                 0
Price Of Sculpture       0
Base Shipping Price      0
International            0
Express Shipment         0
Installation Included    0
Transport                0
Fragile                  0
Customer Information     0
Remote Location          0
Scheduled Date           0
Delivery Date            0
Customer Location        0
Cost                     0
dtype: int64

In [20]:
test.isnull().sum()

Customer Id              0
Artist Name              0
Artist Reputation        0
Height                   0
Width                    0
Weight                   0
Material                 0
Price Of Sculpture       0
Base Shipping Price      0
International            0
Express Shipment         0
Installation Included    0
Transport                0
Fragile                  0
Customer Information     0
Remote Location          0
Scheduled Date           0
Delivery Date            0
Customer Location        0
dtype: int64

**Data Modelling**

In [21]:
train['Scheduled Date']=pd.to_datetime(train['Scheduled Date'],format='%m/%d/%y')
train['Delivery Date']=pd.to_datetime(train['Delivery Date'],format='%m/%d/%y')

In [22]:
train['S_month']=train['Scheduled Date'].dt.month
train['S_year']=train['Scheduled Date'].dt.year
train['S_Day']=train['Scheduled Date'].dt.day

In [23]:
train['D_month']=train['Delivery Date'].dt.month
train['D_year']=train['Delivery Date'].dt.year
train['D_Day']=train['Delivery Date'].dt.day

In [24]:
train.drop(['Scheduled Date','Delivery Date'],1,inplace=True)

In [25]:
test['Scheduled Date']=pd.to_datetime(test['Scheduled Date'],format='%m/%d/%y')
test['Delivery Date']=pd.to_datetime(test['Delivery Date'],format='%m/%d/%y')

In [26]:
test['S_month']=test['Scheduled Date'].dt.month
test['S_year']=test['Scheduled Date'].dt.year
test['S_Day']=test['Scheduled Date'].dt.day

In [27]:
test['D_month']=test['Delivery Date'].dt.month
test['D_year']=test['Delivery Date'].dt.year
test['D_Day']=test['Delivery Date'].dt.day

In [28]:
test.drop(['Scheduled Date','Delivery Date'],1,inplace=True)

In [29]:
train_Y = train['Cost']
train_X = train.drop(['Cost'], axis=1)


In [30]:
train_X=train.drop(['Customer Id'],axis=1)

In [31]:
def divideFeatures(df):
    numerical_features = df.select_dtypes(include=[np.number])
    categorical_features = df.select_dtypes(include=[np.object])
    return numerical_features, categorical_features

In [32]:
continuous_data, categorical_data = divideFeatures(train_X)
categorical_data[:5]

,Artist Name,Material,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location,Customer Location
0,Billy Jenkins,Brass,Yes,Yes,No,Airways,No,Working Class,No,"New Michelle, OH 50777"
1,Jean Bryant,Brass,No,No,No,Roadways,No,Working Class,No,"New Michaelport, WY 12072"
2,Laura Miller,Clay,No,No,No,Roadways,Yes,Working Class,Yes,"Bowmanshire, WA 19241"
3,Robert Chaires,Aluminium,No,No,No,missingTransport,No,Wealthy,Yes,"East Robyn, KY 86375"
4,Rosalyn Krol,Aluminium,Yes,Yes,Yes,Airways,No,Working Class,No,"Aprilside, PA 52793"


In [33]:
df_cat=train_X[['Artist Name','Material','International','Express Shipment','Installation Included','Customer Location','Remote Location','Customer Information','Fragile','Transport']]

In [34]:
df_cat.head()

,Artist Name,Material,International,Express Shipment,Installation Included,Customer Location,Remote Location,Customer Information,Fragile,Transport
0,Billy Jenkins,Brass,Yes,Yes,No,"New Michelle, OH 50777",No,Working Class,No,Airways
1,Jean Bryant,Brass,No,No,No,"New Michaelport, WY 12072",No,Working Class,No,Roadways
2,Laura Miller,Clay,No,No,No,"Bowmanshire, WA 19241",Yes,Working Class,Yes,Roadways
3,Robert Chaires,Aluminium,No,No,No,"East Robyn, KY 86375",Yes,Wealthy,No,missingTransport
4,Rosalyn Krol,Aluminium,Yes,Yes,Yes,"Aprilside, PA 52793",No,Working Class,No,Airways


In [35]:
continuous_data[:5]

,Artist Reputation,Height,Width,Weight,Price Of Sculpture,Base Shipping Price,Cost,S_month,S_year,S_Day,D_month,D_year,D_Day
0,0.26,17.0,6.000000,4128.000000,13.91,16.27,5.646471,6,2015,7,6,2015,3
1,0.28,3.0,3.000000,61.000000,6.83,15.00,5.074924,3,2017,6,3,2017,5
2,0.07,8.0,5.000000,237.000000,4.96,21.18,5.038834,3,2015,9,3,2015,8
3,0.12,9.0,9.617647,400694.821918,5.81,16.31,5.082398,5,2015,24,5,2015,20
4,0.15,17.0,6.000000,324.000000,3.18,11.94,5.070350,12,2016,18,12,2016,14


In [36]:
df_num = train_X.drop(['Cost','Artist Name','Material','International','Express Shipment','Installation Included','Customer Location','Remote Location','Customer Information','Fragile','Transport'],1)

In [37]:
df_num[:5]

,Artist Reputation,Height,Width,Weight,Price Of Sculpture,Base Shipping Price,S_month,S_year,S_Day,D_month,D_year,D_Day
0,0.26,17.0,6.000000,4128.000000,13.91,16.27,6,2015,7,6,2015,3
1,0.28,3.0,3.000000,61.000000,6.83,15.00,3,2017,6,3,2017,5
2,0.07,8.0,5.000000,237.000000,4.96,21.18,3,2015,9,3,2015,8
3,0.12,9.0,9.617647,400694.821918,5.81,16.31,5,2015,24,5,2015,20
4,0.15,17.0,6.000000,324.000000,3.18,11.94,12,2016,18,12,2016,14


In [38]:
df_cat['cmboin'] = df_cat[df_cat.columns[0:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
df_cat.cmboin[1]

'Jean Bryant,Brass,No,No,No,New Michaelport, WY 12072,No,Working Class,No,Roadways'

In [40]:
X_cnvrt=df_cat.cmboin

In [41]:
import regex as re
model = []
for i in range(0, train_X.shape[0]):
    data = re.sub(r'\W', ' ', str(X_cnvrt[i]))
    data = data.lower()
    data = re.sub(r'^br$', ' ', data)
    data = re.sub(r'\s+br\s+',' ',data)
    data = re.sub(r'\s+[a-z]\s+', ' ',data)
    data = re.sub(r'^b\s+', '', data)
    data = re.sub(r'\s+', ' ', data)
    model.append(data)  

In [42]:
model[1]

'jean bryant brass no no no new michaelport wy 12072 no working class no roadways'

In [43]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [45]:
vector = TfidfVectorizer(max_features = 25, min_df = 1, max_df = 0.99, stop_words = stopwords.words('english'))
X_cnvrt = vector.fit_transform(model).toarray()

In [46]:
X_cnvrt

array([[0.37076607, 0.        , 0.        , ..., 0.        , 0.21234782,
        0.3932118 ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.23302376,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.27001662,
        0.49999912],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.37430139],
       [0.        , 0.        , 0.        , ..., 0.        , 0.26809176,
        0.4964348 ],
       [0.39433419, 0.        , 0.        , ..., 0.        , 0.22584592,
        0.20910335]])

In [47]:
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler()
df_num=sc.fit_transform(df_num,1)

In [48]:
df_num

array([[0.26      , 0.2       , 0.08333333, ..., 0.45454545, 0.        ,
        0.06666667],
       [0.28      , 0.        , 0.02083333, ..., 0.18181818, 0.5       ,
        0.13333333],
       [0.07      , 0.07142857, 0.0625    , ..., 0.18181818, 0.        ,
        0.23333333],
       ...,
       [0.68      , 0.22857143, 0.125     , ..., 0.27272727, 1.        ,
        0.26666667],
       [0.02      , 0.42857143, 0.14583333, ..., 0.18181818, 1.        ,
        0.36666667],
       [0.18      , 0.38571429, 0.22916667, ..., 1.        , 0.25      ,
        0.13333333]])

In [49]:
X_num=df_num

In [50]:
X=np.concatenate((X_num,X_cnvrt),1)

In [51]:
X1 = pd.DataFrame(X)

In [52]:
X1[:5]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
0,0.26,0.200000,0.083333,3.497901e-05,2.853163e-05,0.069682,0.454545,0.00,0.200000,0.454545,0.00,0.066667,0.370766,0.000000,0.0,0.495095,0.0,0.212348,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.610422,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.212348,0.393212
1,0.28,0.000000,0.020833,4.918261e-07,1.001614e-05,0.055568,0.181818,0.50,0.166667,0.181818,0.50,0.133333,0.000000,0.000000,0.0,0.543301,0.0,0.233024,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.669858,0.0,0.0,0.384076,0.0,0.0,0.0,0.000000,0.0,0.0,0.233024,0.000000
2,0.07,0.071429,0.062500,1.984264e-06,5.125755e-06,0.124250,0.181818,0.00,0.266667,0.181818,0.00,0.233333,0.000000,0.000000,0.0,0.000000,0.0,0.270017,0.637271,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.445048,0.0,0.0,0.0,0.000000,0.0,0.0,0.270017,0.499999
3,0.12,0.085714,0.158701,3.397770e-03,7.348659e-06,0.070127,0.363636,0.00,0.766667,0.363636,0.00,0.633333,0.000000,0.500642,0.0,0.000000,0.0,0.000000,0.000000,0.621174,0.0,0.0,0.0,0.0,0.418492,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.385877,0.0,0.0,0.000000,0.198654
4,0.15,0.200000,0.083333,2.722003e-06,4.707326e-07,0.021560,1.000000,0.25,0.566667,1.000000,0.25,0.433333,0.409184,0.546821,0.0,0.000000,0.0,0.234351,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.234351,0.650934


In [53]:
test_X = test


In [54]:
test_X[:5]

,Customer Id,Artist Name,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location,Customer Location,S_month,S_year,S_Day,D_month,D_year,D_Day
0,fffe3400310033003300,James Miller,0.35,53.0,18.0,871.0,Wood,5.98,19.11,Yes,Yes,No,Airways,No,Working Class,No,"Santoshaven, IA 63481",7,2017,3,7,2017,6
1,fffe3600350035003400,Karen Vetrano,0.67,7.0,4.0,108.0,Clay,6.92,13.96,No,No,No,Roadways,Yes,Working Class,No,"Ericksonton, OH 98253",5,2016,2,5,2016,2
2,fffe3700360030003500,Roseanne Gaona,0.61,6.0,5.0,97.0,Aluminium,4.23,13.62,Yes,No,No,Airways,No,Working Class,No,APO AP 83453,1,2018,4,1,2018,6
3,fffe350038003600,Todd Almanza,0.14,15.0,8.0,757.0,Clay,6.28,23.79,No,Yes,No,Roadways,Yes,Wealthy,No,"Antonioborough, AL 54778",9,2017,14,9,2017,17
4,fffe3500390032003500,Francis Rivero,0.63,10.0,4.0,1673.0,Marble,4.39,17.83,No,Yes,Yes,Roadways,No,Working Class,Yes,"Lake Frances, LA 03040",12,2017,3,12,2017,2


In [55]:
test_X=test.drop(['Customer Id'],axis=1)

In [56]:
test_X[:5]

,Artist Name,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location,Customer Location,S_month,S_year,S_Day,D_month,D_year,D_Day
0,James Miller,0.35,53.0,18.0,871.0,Wood,5.98,19.11,Yes,Yes,No,Airways,No,Working Class,No,"Santoshaven, IA 63481",7,2017,3,7,2017,6
1,Karen Vetrano,0.67,7.0,4.0,108.0,Clay,6.92,13.96,No,No,No,Roadways,Yes,Working Class,No,"Ericksonton, OH 98253",5,2016,2,5,2016,2
2,Roseanne Gaona,0.61,6.0,5.0,97.0,Aluminium,4.23,13.62,Yes,No,No,Airways,No,Working Class,No,APO AP 83453,1,2018,4,1,2018,6
3,Todd Almanza,0.14,15.0,8.0,757.0,Clay,6.28,23.79,No,Yes,No,Roadways,Yes,Wealthy,No,"Antonioborough, AL 54778",9,2017,14,9,2017,17
4,Francis Rivero,0.63,10.0,4.0,1673.0,Marble,4.39,17.83,No,Yes,Yes,Roadways,No,Working Class,Yes,"Lake Frances, LA 03040",12,2017,3,12,2017,2


In [57]:
df_cat_test =test_X[['Artist Name','Material','International','Express Shipment','Installation Included','Customer Location','Remote Location','Customer Information','Fragile','Transport']]

In [58]:
df_num_test = test_X.drop(['Artist Name','Material','International','Express Shipment','Installation Included','Customer Location','Remote Location','Customer Information','Fragile','Transport'],1)

In [59]:
df_cat_test['cmboin'] = df_cat[df_cat_test.columns[0:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [60]:
df_cat_test.cmboin[1]

'Jean Bryant,Brass,No,No,No,New Michaelport, WY 12072,No,Working Class,No,Roadways'

In [61]:
X_cnvrt_t=df_cat_test.cmboin

In [62]:
import regex as re
model_t = []
for i in range(0, test.shape[0]):
    data = re.sub(r'\W', ' ', str(X_cnvrt_t[i]))
    data = data.lower()
    data = re.sub(r'^br$', ' ', data)
    data = re.sub(r'\s+br\s+',' ',data)
    data = re.sub(r'\s+[a-z]\s+', ' ',data)
    data = re.sub(r'^b\s+', '', data)
    data = re.sub(r'\s+', ' ', data)
    model_t.append(data)  

In [63]:
model_t[1]

'jean bryant brass no no no new michaelport wy 12072 no working class no roadways'

In [64]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [65]:
vector = TfidfVectorizer(max_features = 25, min_df = 1, max_df = 0.99, stop_words = stopwords.words('english'))
X_cnvrt_t = vector.fit_transform(model_t).toarray()

In [66]:
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler()
df_num_test=sc.fit_transform(df_num_test,1)

In [67]:
X_num_test=df_num_test

In [68]:
X_t=np.concatenate((X_num_test,X_cnvrt_t),1)

In [69]:
X1_t = pd.DataFrame(X_t)

In [70]:
X1_t[:5]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
0,0.35,0.806452,0.347826,0.000013,0.000013,0.101245,0.545455,0.50,0.066667,0.545455,0.6,0.166667,0.0,0.371513,0.000000,0.496768,0.0,0.211584,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.608671,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.211584,0.393933
1,0.67,0.064516,0.043478,0.000002,0.000017,0.044010,0.363636,0.25,0.033333,0.363636,0.4,0.033333,0.0,0.000000,0.000000,0.544585,0.0,0.231950,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.667259,0.0,0.0,0.38806,0.0,0.0,0.0,0.000000,0.0,0.0,0.231950,0.000000
2,0.61,0.048387,0.065217,0.000001,0.000005,0.040231,0.000000,0.75,0.100000,0.000000,0.8,0.166667,0.0,0.000000,0.000000,0.000000,0.0,0.266749,0.641772,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.44628,0.0,0.0,0.0,0.000000,0.0,0.0,0.266749,0.496641
3,0.14,0.193548,0.130435,0.000012,0.000014,0.153256,0.727273,0.50,0.433333,0.727273,0.6,0.533333,0.0,0.000000,0.498150,0.000000,0.0,0.000000,0.000000,0.622536,0.0,0.0,0.0,0.0,0.414444,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.391024,0.0,0.0,0.000000,0.199076
4,0.63,0.112903,0.043478,0.000026,0.000006,0.087019,1.000000,0.50,0.066667,1.000000,0.6,0.033333,0.0,0.410357,0.544404,0.000000,0.0,0.233707,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.233707,0.652682


In [71]:
from sklearn.model_selection import train_test_split

In [72]:

X_train,X_test,y_train,y_test=train_test_split(X,train_Y,test_size=0.1,random_state=42)

In [73]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_jobs = -1, random_state = 42, n_estimators = 500, verbose = 1)
regr.fit(X_train, y_train)
regr.score(X_train,y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   22.9s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   25.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:    0.5s finished


0.9942464061776136

In [74]:
from sklearn import metrics
pred1 = regr.predict(X_test) 
rmse = np.sqrt(metrics.mean_squared_error(y_test, pred1)) 
print("RMSE : % f" %(rmse)) 

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:    0.2s finished


RMSE :  0.326391


In [75]:
metrics.r2_score(y_test,pred1)*100

96.14604151319331

In [76]:
import lightgbm as lgb
train = lgb.Dataset(X_train, label=y_train)
test = lgb.Dataset(X_test, label=y_test)

In [77]:
param = {'objective': 'regression',
         'boosting': 'gbdt',  
         'metric': 'l2_root',
         'learning_rate': 0.01, 
         'num_iterations': 10000,
         'min_data_in_leaf': 15,
         'bagging_fraction': 0.90,
         'bagging_freq': 2,
         'bagging_seed': 3,
         'feature_fraction': 0.99,
         'feature_fraction_seed': 2,
         'early_stopping_round': 2000,
         'max_bin': 500
         }

In [78]:
gbm = lgb.LGBMRegressor(**param)

In [79]:
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds = 100)

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Streaming output truncated to the last 5000 lines.
[321]	valid_0's rmse: 0.330338	valid_0's l1: 0.220632
[322]	valid_0's rmse: 0.330037	valid_0's l1: 0.220341
[323]	valid_0's rmse: 0.32963	valid_0's l1: 0.220004
[324]	valid_0's rmse: 0.329264	valid_0's l1: 0.219691
[325]	valid_0's rmse: 0.328952	valid_0's l1: 0.219374
[326]	valid_0's rmse: 0.328627	valid_0's l1: 0.219058
[327]	valid_0's rmse: 0.328313	valid_0's l1: 0.218782
[328]	valid_0's rmse: 0.327958	valid_0's l1: 0.218478
[329]	valid_0's rmse: 0.327685	valid_0's l1: 0.218253
[330]	valid_0's rmse: 0.327388	valid_0's l1: 0.218027
[331]	valid_0's rmse: 0.32717	valid_0's l1: 0.217819
[332]	valid_0's rmse: 0.326962	valid_0's l1: 0.217572
[333]	valid_0's rmse: 0.326682	valid_0's l1: 0.217354
[334]	valid_0's rmse: 0.326402	valid_0's l1: 0.217111
[335]	valid_0's rmse: 0.326003	valid_0's l1: 0.216741
[336]	valid_0's rmse: 0.325626	valid_0's l1: 0.216392
[337]	valid_0's rmse: 0.325233	valid_0's l1: 0.216017
[338]	valid_0's rmse: 0.32485	val

LGBMRegressor(bagging_fraction=0.9, bagging_freq=2, bagging_seed=3,
              boosting='gbdt', boosting_type='gbdt', class_weight=None,
              colsample_bytree=1.0, early_stopping_round=2000,
              feature_fraction=0.99, feature_fraction_seed=2,
              importance_type='split', learning_rate=0.01, max_bin=500,
              max_depth=-1, metric='l2_root', min_child_samples=20,
              min_child_weight=0.001, min_data_in_leaf=15, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_iterations=10000, num_leaves=31,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, ...)

In [80]:
pred1 = gbm.predict(X_test, num_iteration = gbm.best_iteration_) 
rmse = np.sqrt(metrics.mean_squared_error(y_test, pred1)) 
print("RMSE : % f" %(rmse)) 
print(metrics.r2_score(y_test,pred1)*100)

RMSE :  0.283182
97.0989008858246


In [81]:
gb_pred=gbm.predict(X_t)

In [82]:
gb_pred

array([ 5.94181829,  6.47872174,  5.32463488, ...,  6.03234806,
        6.54143449, 13.17365958])

In [83]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_log_error

errlgb = []
y_pred_totlgb = []

fold = KFold(n_splits = 10, shuffle=True, random_state=42)

for train_index, test_index in fold.split(X1):
    X_train, X_test = X1.loc[train_index], X1.loc[test_index]
    y_train, y_test = train_Y[train_index], train_Y[test_index]

    lgbm = LGBMRegressor(**param)
    lgbm.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0, early_stopping_rounds=200)

    y_pred_lgbm = lgbm.predict(X_test)
    print("RMSE LGBM: ",  np.sqrt(metrics.mean_squared_error(y_test, y_pred_lgbm)))
    print("R2 LGBM: ",  metrics.r2_score(y_test,y_pred_lgbm)*100)
    print("**********************")
    errlgb.append(np.sqrt(metrics.mean_squared_error(y_test, y_pred_lgbm)))
    p = lgbm.predict(X_t)
    y_pred_totlgb.append(p)

RMSE LGBM:  0.28644799805096477
R2 LGBM:  97.03160735354078
**********************
RMSE LGBM:  0.33640653171659746
R2 LGBM:  95.79656218519584
**********************
RMSE LGBM:  0.2752944390229765
R2 LGBM:  97.07303461442247
**********************
RMSE LGBM:  0.2904141058252399
R2 LGBM:  97.1252181984568
**********************
RMSE LGBM:  0.27793190565115444
R2 LGBM:  97.34506170301376
**********************
RMSE LGBM:  0.26589129070395007
R2 LGBM:  96.85563165799012
**********************
RMSE LGBM:  0.2949959832904927
R2 LGBM:  96.6756722140185
**********************
RMSE LGBM:  0.3148956921803661
R2 LGBM:  96.76986667808679
**********************
RMSE LGBM:  0.250185825079914
R2 LGBM:  97.7142627231373
**********************
RMSE LGBM:  0.28927433371744243
R2 LGBM:  96.92290465733076
**********************


In [84]:
np.mean(errlgb,0)

0.28817381052390983

In [85]:
lgbm_final = np.mean(y_pred_totlgb,0)
lgbm_final = np.exp(lgbm_final)
lgbm_final

array([3.70407838e+02, 6.28118385e+02, 2.10390063e+02, ...,
       3.94922835e+02, 7.30381686e+02, 5.27376706e+05])

In [86]:
pred_regr = regr.predict(X_t)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:    0.3s finished


In [87]:
result=np.exp(lgbm_final)

In [90]:
data = {
        'Customer Id': test['Customer Id'],
        'Cost': lgbm_final} 

In [91]:
sub = pd.DataFrame(data = data)

In [92]:
sub.to_csv('result_regr-2.csv',index=False)

In [93]:
sub

,Customer Id,Cost
0,fffe3400310033003300,370.407838
1,fffe3600350035003400,628.118385
2,fffe3700360030003500,210.390063
3,fffe350038003600,219.416405
4,fffe3500390032003500,374.092227
...,...,...
3495,fffe3200300037003100,869.749006
3496,fffe390031003100,5457.863657
3497,fffe3200350031003700,394.922835
3498,fffe3100380031003000,730.381686
